In [2]:

  import mraa  
2 import time 
3 import socket   #for sockets 
4 import sys  #for exit 
5 
 
6 INTERVAL = .05                   # Sound Signature Sample time 
7 SENDMSG_INTERVAL =60           # Minimum time between sending something to cloud 
8 
 
9 LED_GPIO = 5                   # The LED pin 
10 TRIGGER_GPIO = 6               # The TRIGGER GPIO 
11 led = mraa.Gpio(LED_GPIO)      # Get the LED pin object 
12 led.dir(mraa.DIR_OUT)          # Set the direction as output 
13 trig = mraa.Gpio(TRIGGER_GPIO)   # Get the TRIGGER pin object 
14 trig.dir(mraa.DIR_IN)           # Set the direction as input 
15 
 
16 ledState = False               # LED is off to begin with 
17 led.write(0) 
18 
 
19 
 
20 def getTriggerf(): 
21     """ This function operates for minimal send interval """ 
22     t = time.time() 
23     next_sample_time = t + SENDMSG_INTERVAL 
24      
25     while (1): 
26         t = time.time() 
27         if t > next_sample_time: 
28             print 'Timed Out' 
29             return (0) 
30         if (trig.read() != 0): 
31             # No trigger detected 
32             continue 
33         else: 
34             # Detected a trigger 
35             return (1) 
36 
 
37 
 
38 def getSignature(): 
39     """ This function determines if a valid shot is fired """                 
40     t = time.time()  
41     next_sample_time = t + INTERVAL 
42     lowCt=0 
43     highCt=0 
44     prevstate=0 
45     toggleCt=1 
46                  
47     while True: 
48         t = time.time()   
49         if (trig.read() != prevstate): 
50             toggleCt += 1 
51             prevstate= trig.read() 
52         else: 
53 
 
54             if t > next_sample_time: 
55                 print highCt 
56                 print lowCt 
57                 print toggleCt 
58                 shotdata=[] 
59                 shotdata.append (highCt) 
60                 shotdata.append (lowCt) 
61                 shotdata.append (toggleCt) 
62                 return shotdata 
63 
 
64                      
65         if trig.read() == 1: 
66              # valid highpulse 
67              highCt += 1 
68              continue 
69                      
70         else: 
71             # Detected a click 
72             lowCt += 1 
73             continue 
74              
75              
76           
77 
 
78 if __name__ == '__main__': 
79     host = '127.0.0.1' 
80     port = 41234 
81     msg = '{"n": "temp", "v": 1.0}' 
82     nullmsg = '{"n": "temp", "v": 0.0}' 
83     shotCt=0 
84     # initialize our socket... 
85     try: 
86         s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
87     except socket.error: 
88         print 'Failed to create socket' 
89         sys.exit() 
90 
 
91     while 1: 
92         # wait until trigger or timed out 
93         lowtrig = getTriggerf() 
94         print 'Sound Detected or Timed Out' 
95   
96         if lowtrig == 0: 
97                   
98             try: 
99                 #Set the whole string 
100                 s.sendto(nullmsg, (host, port)) 
101                 print 'Null Message Sent' 
102                 print nullmsg 
103 
 
104             except socket.error, msg: 
105                 print 'Error Code : ' + str(msg[0]) + ' Message ' + msg[1] 
106                 sys.exit()  
107             continue 
108         else:     
109             shotdata =getSignature () 
110             print 'Signature Detected' 
111             print shotdata 
112 
 
113             if shotdata[1] > 5 < 100 and shotdata[2] > 5 < 50 : 
114                 try : 
115                     #Set the whole string 
116                     s.sendto(msg, (host, port)) 
117                     shotCt += 1 
118 		    print 'Message Sent', shotCt 
119 
 
120                 except socket.error, msg: 
121                     print 'Error Code : ' + str(msg[0]) + ' Message ' + msg[1] 
122                     sys.exit() 
123 
 
124         # Button click, detected, now toggle the LED 
125 #        if ledState == True: 
126 #            led.write(1) 
127 #            ledState = False 
128 #        else: 
129 #            led.write(0) 
130 #            ledState = True 
131 
 
132 #    time.sleep(0.005) 


IndentationError: unexpected indent (<ipython-input-2-ec01805ef39c>, line 2)